In [ ]:
#here everyone pastes their individual working parts after working on them in their personal branch


<!--TABLE OF CONTENTS-->
Contents:
- [Introduction](#Introduction)
  - [What business problem are you solving?](#What-business-problem-are-you-solving?)
  - [What is the machine learning problem that you are solving?](#What-is-the-machine-learning-problem-that-you-are-solving?)
- [Data exploration and preparation](#Data-exploration-and-preparation)
- [Feature Engineering and ex ante Feature Selection](#Feature-Engineering-and-ex-ante-Feature-Selection)
- [Modeling](#Modeling)
  - [Linear Models](#Linear-Models)
    - [Ridge](#Ridge)
    - [Lasso](#Lasso)
  - [Tree Models](#Tree-Models)
    - [XGBoost](#XGBoost)
    - [CatBoost](#CatBoost)
    - [Random Forest](#Random-Forest)
  - [Classification Models](#Classification-Models)
    - [KNN](#KNN)
  - [Neural Network](#Neural-Network)
- [Model Evaluation](#Model-Evaluation)
- [Model Interpretation](#Model-Interpretation)

## Introduction
### Group number:
### Group members: Timm Vordermark
### Student IDs: 65983
### Project name:

## What business problem are you solving?
- Please state clearly what business problem are you solving. (one sentence)
- Elaborate why is this a relevant problem, and what can you do with the model output to create business value, i.e., how is the model output actionable. (2-3 paragraphs)

## What is the machine learning problem that you are solving?
- Please state clearly what is the ML problem. 
- If applicable state your target.

## Data exploration and preparation 

- How many data instances do you have?
- Do you have duplicates?
- How many features? What type are they?
- If they are categorical, what categories they have, what is their frequency?
- If they are numerical, what is their distribution?
- Do you have outliers, and do you need to do anything about them?
- What is the distribution of the target variable?
- If you have a target, you can also check the relationship between the target and the variables.
- Do you have missing data? If yes, how are you going to handle it?
- Can you use the features in their original form, or do you need to alter them in some way?
- What have you learned about your data? Is there anything that can help you in feature engineering or modeling?


In the following section we perform comprehensive cleaning an preprocessing on the given financial dataset. First,  the Date column is standadized to datetime format and rows with missing date values are removed. We also drop rows that have missing values in both Brand_Name and Ticker, as these are essential identifiers. The code then standardizes text by converting Brand_Name to lowercase and Ticker to uppercase. We built dictionaries to impute missing Ticker or Brand_Name values based on the other, using the existing mappings from Ticker/Brand_Name. Sparse event columns like Dividends and Stock Splits are filled with 0, assuming no events. For Industry_Tag and Country, the most common value per brand is used to fill missing or inconsistent entries. Finally, columns like Open, High, and Low—which correlate closely with the target Close price—are dropped to avoid data leakage, along with Brand_Name (redundant with Ticker). The dataset is further cleaned by dropping rows with missing target (Close) and feature (Volume) values to ensure high-quality inputs for modeling.

# Begründung Volume

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder,MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.inspection import permutation_importance
import shap
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor, plot_tree
from scipy.stats import uniform as sp_uniform
from catboost import CatBoostRegressor
import pickle
from sklearn.ensemble import RandomForestRegressor

c:\Users\timmv\anaconda3\envs\ml2025\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv('pr13_stocks (1).csv', index_col=0)


In [ ]:
#General Information about the Dataset
print(df.shape) 
print(df.dtypes) 
print(df.head())
sns.histplot(data=df, x='Close', bins='auto', kde=True)
plt.show()
print('Number of duplicates:', np.sum(df.duplicated()))
print('Number of Missing Values:', df.isnull().sum().sum())
#General Descriptives about the Dataset
df.info()
df.describe()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], utc=True)
#drop the 16 rows without any date data
df = df.dropna(subset=['Date'])
#drop the row without both Brand_name and Ticker as we cannot say which company this entry refers to
df = df[~(df['Brand_Name'].isna() & df['Ticker'].isna())].copy()

df['Brand_Name'] = df['Brand_Name'].str.lower()
df['Ticker'] = df['Ticker'].str.upper()

brand_to_ticker = df.dropna(subset=['Brand_Name', 'Ticker'])\
                        .drop_duplicates(subset=['Brand_Name'])\
                        .set_index('Brand_Name')['Ticker'].to_dict()

ticker_to_brand = df.dropna(subset=['Brand_Name', 'Ticker'])\
                        .drop_duplicates(subset=['Ticker'])\
                        .set_index('Ticker')['Brand_Name'].to_dict()

    # Fill missing Ticker using Brand_Name
df.loc[df['Ticker'].isna() & df['Brand_Name'].notna(), 'Ticker'] = (
        df.loc[df['Ticker'].isna() & df['Brand_Name'].notna(), 'Brand_Name']
        .map(brand_to_ticker)
    )

    # Fill missing Brand_Name using Ticker
df.loc[df['Brand_Name'].isna() & df['Ticker'].notna(), 'Brand_Name'] = (
    df.loc[df['Brand_Name'].isna() & df['Ticker'].notna(), 'Ticker']
    .map(ticker_to_brand))



#As dividend payments and stock splits are sparse, event-based features, missing values here are likely because 
#no such event has occured. Therefore, missing values are filled with 0. Also, the occurence of missing values 
#is relatively low here(59 and 458, respectively) compared to the overall dataset size(~100,000)
df['Dividends'] = df['Dividends'].fillna(0.0)
df['Stock Splits'] = df['Stock Splits'].fillna(0.0)




#Imputing industry tag values with the most frequent value for each brand

df['Industry_Tag'] = df['Industry_Tag'].str.strip().str.lower()
industry_map = (
        df.dropna(subset=['Industry_Tag'])
          .groupby('Brand_Name')['Industry_Tag']
          .agg(lambda x: x.mode()[0]) 
          .to_dict()
    )

df['Industry_Tag'] = df.apply(
        lambda row: industry_map[row['Brand_Name']]
        if pd.isna(row['Industry_Tag']) or row['Industry_Tag'] != industry_map.get(row['Brand_Name'])
        else row['Industry_Tag'],
        axis=1
    )

#same for country

df['Country'] = df['Country'].str.strip()

country_map = (
        df.dropna(subset=['Country'])
          .groupby('Brand_Name')['Country']
          .agg(lambda x: x.mode()[0]) 
          .to_dict()
    )

df['Country'] = df.apply(
        lambda row: country_map[row['Brand_Name']]
        if pd.isna(row['Country']) or row['Country'] != country_map.get(row['Brand_Name'])
        else row['Country'],
        axis=1
    )

#drop Open High Low since they give information at the same day as they are not independent of the Close price --> our target
# drop Brand Name as it contains the same info as Ticker
df = df.drop(columns=['Open', 'High', 'Low','Brand_Name'])
#drop all nans in target and the corresponding rows -->hard to predict without target
print(df.shape,'shape before Close dropped')
df = df.dropna(subset=['Close']) #194 rows dropped
print(df.shape,'shape before vol dropped')
#drop all nans in volumne and the corresponding rows
df = df.dropna(subset=['Volume']) #852 rows dropped
print(df.shape,'shape after vol dropped')


#we decided not to exclude outliers, however they are defined, since we expect the models to learn from these 'special' circumstances.


# Outliers Dropped?

In [ ]:
duplicated = df[df.duplicated(subset=['Ticker', 'Date'], keep=False)]
duplicated
#drop all duplicates if the ticker and Date are the same
df = df.drop_duplicates(subset=['Ticker', 'Date'], keep='first')

In [ ]:
#check
print(df.isna().sum())
df

## Feature Engineering and ex ante Feature Selection
Creating good features is probably the most important step in the machine learning process. 
This might involve doing:
- transformations
- aggregating over data points or over time and space, or finding differences (for example: differences between two monthly bills, time difference between two contacts with the client) 
- creating dummy (binary) variables
- discretization

Business insight is very relevant in this process. If it is possible you can also find additional relevant data.

As decribed above the idea behind feature engineering is to create new features that carry more predictive power and help the model to increase its performance.We start by extracting time-related features from the Date column, such as the year, month, day of year, and day of the week, under the assumption that stock performance may follow time patterns. Day_Number is also computed to represent time progression numerically.

In [ ]:
#include new time features, since we expected to find some pattern of the time, e.g. an increase over the years (that we see), or an increase on certain days of the week or the year (cannot observe that).
df['Date'] = pd.to_datetime(df['Date']) 

# Time-based features
df['Year'] = df['Date'].dt.year.astype('int32')
df['Month'] = df['Date'].dt.month.astype('int32')
df['Day'] = df['Date'].dt.dayofyear.astype('int32')
df['DayOfWeek'] = df['Date'].dt.dayofweek.astype('int32')  # 0 = Monday, 6 = Sunday

# Running day number to determine the date on, rather than a datetime type
df['Day_Number'] = (df['Date'] - df['Date'].min()).dt.days
df = df.sort_values(by='Date').reset_index(drop=True)

The dataframe is enriched with lagged features using the add_lags function, which creating shifted versions of the Close price and Volume, as well as the number of days between the current and lagged observations. The idea is that the model can identify short-term historical trends.

In [ ]:
#we wanted to add the past close prices, volumnes and the time distance to the current date in days to the df
def add_lags(df, lags):
    for lag in lags:
        df[f'Close_Lag_{lag}' ] = df.groupby('Ticker' )['Close'].shift(lag)
        df[f'Volume_Lag_{lag}' ] = df.groupby('Ticker')['Volume'].shift(lag)
        df[f'Days_Since_Lag_{lag}' ] = df.groupby('Ticker')['Date'].diff(lag).dt.days
    return df


To include time and price sensitive features we wanted to include a time weighted average and std. This ensures that more recent prices carry more weight, reflecting their likely higher relevance to current market behavior. At the same time simple unweighted rolling mean and std are computed for comparison using the add_simple_mean_std function, which helps to evaluate the effect of time-weighting on predictive features.

In [ ]:
#we wanted to implement a time weighted mean and std to the df. because our initial function took over 30min we improved speed with numba
from numba import njit

#calculate the weighted mean and std using numba (dramatic time reduction)
@njit
def weighted_mean_std_numba(close_lags, days_diff):

    if np.sum(days_diff) == 0:
        return np.nan, np.nan

    weight = 1.0/days_diff
    weight_sum = np.sum(weight)


    if weight_sum == 0.0 or np.any(np.isnan(weight)) or np.any(np.isinf(weight)):
        return np.nan, np.nan



    weighted_mean = np.sum(weight* close_lags) /weight_sum
    weighted_var = np.sum(weight*(close_lags- weighted_mean)**2) /weight_sum
    weighted_std = np.sqrt(weighted_var)

    return weighted_mean, weighted_std

def add_weighted_mean_std_numba(df, lookback_periods):
    df = df.sort_values(['Ticker', 'Date']).copy()

    for lookback in lookback_periods:
        weighted_means = np.full(len(df),np.nan)
        weighted_stds = np.full(len(df), np.nan)

        idx = df.index.to_numpy()
        closes = df['Close'].to_numpy()
        dates = pd.to_datetime(df['Date']).values.astype('datetime64[D]')
        tickers = df['Ticker'].to_numpy()

        unique_tickers = np.unique(tickers)

        for ticker in unique_tickers:

            ticker_mask = tickers == ticker
            ticker_indices = np.where(ticker_mask)[0]

            for i in range(lookback, len(ticker_indices)):

                idx_range = ticker_indices[i - lookback:i]
                current_idx = ticker_indices[i]


                close_lags = closes[idx_range]

                date_lags = dates[idx_range]
                current_date = dates[current_idx]
                days_diff = (current_date - date_lags).astype(np.int64)



                if np.any(days_diff == 0) or np.any(np.isnan(close_lags)):
                    continue 

                mean_val, std_val = weighted_mean_std_numba(close_lags , days_diff)
                weighted_means[current_idx] =mean_val
                weighted_stds[current_idx] = std_val

        df[f'Weighted_Mean_{lookback}']= weighted_means
        df[f'Weighted_Std_{lookback}']= weighted_stds

    return df


In [ ]:
#we also wanted to see how simply shifting the lookback period would influence the feature, without taking the timing differnce into account.
def add_simple_mean_std(df, lookback_periods):
    df = df.sort_values(['Ticker', 'Date']).copy()

    for lookback in lookback_periods:

        means =[]
        stds =[]
        for ticker, group in df.groupby('Ticker'):

            group = group.sort_values('Date')
            closes = group['Close_Lag_1'].reset_index(drop=True)
            mean_vals = closes.rolling(window=lookback,min_periods=lookback).mean()
            std_vals = closes.rolling(window=lookback, min_periods=lookback).std()

            means.append(pd.Series(mean_vals.values, index=group.index))
            stds.append(pd.Series(std_vals.values, index=group.index))


        df[f'Simple_Mean_{lookback}'] =pd.concat(means)
        df[f'Simple_Std_{lookback}'] =pd.concat(stds)

    return df

Additional engineered features include daily returns and differences in Close prices, giving us insight into market momentum from the last close price.

In [ ]:
#we added the return of the previous day
def add_returns(df):
    df['return_since_last_entry'] = df['Close_Lag_1']/df['Close_Lag_2'] -1

    return df

In [ ]:
#we added the Close price difference of the previous day
def add_diff(df):
    df['diff_since_last_entry'] = df['Close_Lag_1']-df['Close_Lag_2']

    return df

Binary signal features are also introduced to indicate the presence of dividends and stock splits, rather then just their numerical value. The idea was to help the model detect event-driven price changes, just on the events not on their actual value.

In [ ]:
#since so many rows for dividends and stock splits are zero we wanted to add a signal for it to the df
def add_cat_signals(df, cols):
    for col in cols:
        new_col = f"{col}_signal"
        df[new_col] = (df[col] != 0).astype('int32')
    return df

In [ ]:
df = add_lags(df, [1,2,3,4,5,10,20,30,40,50,60,70,80,90,100])

In [ ]:
df = add_weighted_mean_std_numba(df,[2,3,4,5,10,20,30,40,50,60,70,80,90,100])


In [ ]:
df = add_simple_mean_std(df,[2,3,4,5,10,20,30,40,50,60,70,80,90,100])


In [ ]:
df = add_returns(df)


In [ ]:
df = add_diff(df)


In [ ]:
df = add_cat_signals(df, ['Dividends','Stock Splits'])

In [ ]:
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(df.columns)
df.isna().sum()

We delete the Date column since we have the same information in the Day_number column. Moreover we cannot include the volumne column since this value also captures information that is not avialable before market closing.

In [ ]:
#since date is in the datetime type and the information of time is preserved in the Day_number column we can delete it
df = df.drop('Date', axis=1)
#we need to delete Volumne since it contains information to some extend about the same day. therefore we cannot use it. remeber we still have a lagged volumne in the df
df = df.drop('Volume', axis=1)
# print(df['Stock Splits_signal'].sum())

In [ ]:
# df.to_csv('pr13_all_cols.csv', index=False)

# oder hier nicht samplen sondern das letzte Jahr nehmen?

In [ ]:
#to visualize dependecies between features and grasp their behavior we make the assumption that ~10k rows = 10% of the dataset is sufficient
# df_fs = df.sample(frac=0.1, random_state=42)
df_fs = df[df['Day_Number']>=8036] # we assume that the last time period is more important rather then randomly sampling
print(df[df['Day_Number']==8036]) #first day in 2022 --> 9273 rows ~10%

In [ ]:
filtered_df = df_fs[df_fs['Close'] > 1800]
filtered_df

df_fs = df_fs[df_fs['Ticker'] != 'CMG']

# Outlier CMG ???

These scatterplots visualize Close vs. Year, Month, Day, and DayOfWeek, to explore how Close behaves over time and identify time-related patterns in stock prices. We observe higher Close prices for an increase in year. Moreover we see that several tickers show gernerally higher Close prices.

In [ ]:
#Here we wanted to visualize the behavior of time feature against the target column Close. E.g. find certain days where Close prices are higher then normal

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_fs, x="Close", y="Year",hue="Ticker", legend=False)
plt.title('Close vs Year')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_fs, x="Close", y="Year",hue="Ticker", legend=False)
plt.xlim(0,400)
plt.title('Close vs Year (Zoomed)')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_fs, x="Close", y="Month", hue="Ticker", legend=False)
plt.title('Close vs Month')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_fs, x="Close", y="Day", hue="Ticker", legend=False)
plt.title('Close vs Day')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_fs, x="Close", y="DayOfWeek", hue="Ticker", legend=False)
plt.title('Close vs DayOfWeek')
plt.show()

We plot relationships between numerical features (Close, Dividends, Stock Splits, and their binary signal columns) to see how these variables interact. Since we only have 17 stocksplit in the entire dataset and we sampled 10% we have 0 stocksplits in our subset. For dividends we see higher dividends for higher stock prices. The rest of the features does not really provides more insights.

In [ ]:
# 'pairplot of close, div, stock split, theri signal' remember there is only 17 stocksplits and around 900 dividends
g= sns.pairplot(df_fs[['Close', 'Dividends', 'Stock Splits', 'Dividends_signal', 'Stock Splits_signal','Ticker']], hue = 'Ticker', diag_kind="kde", corner=True)
g._legend.remove()
plt.show()

We now explore how categorical features (Country, Industry, and Ticker) relate to Close and want to observe their distribution. We see higher Close prices for usa, certain tickers (ADBE,NFLX etc.) and industries like technology and entertainment.

In [ ]:
#in the following we visualize the categorical columns Ticker , Country and Industry against the Close price
plt.figure(figsize=(10, 6))
sns.stripplot(x='Country', y='Close', data=df_fs, jitter=True, palette='deep')
plt.title('Country vs Close')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(10, 6))
sns.stripplot(x='Industry_Tag', y='Close', data=df_fs, jitter=True, palette='muted')
plt.title('Industry vs Close')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(10, 6))
sns.stripplot(x='Ticker', y='Close', data=df_fs, jitter=True, palette='bright')
plt.title('Ticker vs Close')
plt.xticks(rotation=90)
plt.show()


Are their any interactions between the categroical features? We basically observe the same trends we saw above.

In [ ]:
#Since the pairplot only support numerical varibles we decided to encode our categorical columns in a ordinal way, to be able to plot a pairplot
# Encode categorical features
plt.figure(figsize=(35, 35))
encoder = OrdinalEncoder()
encoded_features = encoder.fit_transform(df_fs[['Ticker', 'Industry_Tag','Country']])
encoded_df = pd.DataFrame(encoded_features , columns=['Ticker','Industry_Tag', 'Country'])
encoded_df['Close'] = df_fs['Close' ].values
g = sns.pairplot(encoded_df, diag_kind="kde", corner=True)

#from ordinal label axis back to names--> due to the number of ticker/industies --> not all names displayable
original_labels = df_fs[['Ticker', 'Industry_Tag','Country']]
for i, col in enumerate(['Ticker', 'Industry_Tag','Country']):
    mapping = dict(zip(
        encoded_df[col].unique(),
        original_labels[col].astype(str).unique()))
    for ax in g.axes.flat:
        if ax is not None:
            # Xlabels
            if ax.get_xlabel() == col:
                ticks = ax.get_xticks()
                labels = [mapping.get(tick, '') for tick in ticks]
                ax.set_xticklabels(labels, rotation=45)
            #ylabels
            if ax.get_ylabel() == col:
                ticks = ax.get_yticks()
                labels = [mapping.get(tick, '') for tick in ticks]
                ax.set_yticklabels(labels, rotation=0)
plt.tight_layout()
plt.show()

We also wanted to visualize the dependece of Close by return (relative change) and price difference (absolute change) from the last day. We find that returns are normally/t-student distributed, as expected (with some outlying tickers), while the price difference since the last entry is also centered around 0 with some tickers coloring the outer parts of the distribution.

In [ ]:
g = sns.pairplot(df_fs[['return_since_last_entry','diff_since_last_entry','Close', 'Ticker']], hue = "Ticker", diag_kind="kde", corner=True)
g._legend.remove()
plt.show()

As we expect that the most recent features have the most impact on the Close price we plotted them individually and observed a strong linear relationship between the last Close price and our target. Volume and the number of days since the last entry do not seem to give much information.

In [ ]:
g = sns.pairplot(df_fs[['Close_Lag_1','Volume_Lag_1','Days_Since_Lag_1','Close', 'Ticker']], hue = "Ticker", diag_kind="kde", corner=True)
g._legend.remove()
plt.show()

Moreover we wanted to see the influence of the weighting in comparison to the simple mean and std for different lookback periods. For reasons of simplicity and clarity we are only showing the lookback period = 2. What we can observe in these pairplots, that the weigthing seems to increase the linearity of the model, since the distribution of mean and std seems to be less spread if the prices are time weighted. Again Volume and the number of days since the last entry seem not to provide much insights.

In [ ]:
lookback_list = [2]#,3,4,5,10,20,30,40,50,60,70,80,90,100] # we looked at all added features but for structure we just show lag 2 here. uncomment the remaining lags to see all
for lookback in lookback_list:
    print('Status: Lag', lookback)
    feature_list = f'Simple_Mean_{lookback}'
    g = sns.pairplot(df_fs[[f'Close_Lag_{lookback}',f'Volume_Lag_{lookback}',f'Days_Since_Lag_{lookback}',f'Simple_Mean_{lookback}',f'Simple_Std_{lookback}',f'Weighted_Mean_{lookback}',f'Weighted_Std_{lookback}','Close', 'Ticker']], hue = "Ticker", diag_kind="kde", corner=True)
    g._legend.remove()
    plt.show() 

# what we see is that the weighted average shows cleaner linear distribution to the Close price, also the std is less spread --> prefer weighted over simple
# we observe that Days Since and volumne seem not to carry much information. Still left the last Volume Day Since to check later with correlation and mututual information

As a result we dropped all simple mean and std column since they would just include more noise than the weighted mean and std for the same lookback period. For Volumne and number of days since the last entry we decided to keep the last entry since this might be the most important out of all the volumne and number of days since the last entry to check their importance later.

In [ ]:
df_fs =df_fs.drop(columns=[col for  col in df_fs.columns if 'Simple' in col]) #all simple dropped
days_cols = [col  for col in df_fs.columns if 'Days_Since' in col]
df_fs = df_fs.drop(columns=days_cols[1:] ) # all but the first day since dropped
volume_cols = [col for col in df_fs.columns if 'Volume_Lag' in col]
df_fs =df_fs.drop( columns=volume_cols[1:]) # all but the first volumne dropped

In [ ]:
#determine categorical and numerical columns here

column_types = pd.DataFrame({
    'Column Name': df_fs.columns,
    'Data Type': df_fs.dtypes.values
})
print(column_types)

numerical_cols = df_fs.select_dtypes(include='float64').columns.tolist() 
numerical_cols.append('Day_Number')

all_columns = df_fs.columns.tolist()

categorical_cols = list(set(all_columns) - set(numerical_cols))

print(categorical_cols)
print(numerical_cols)


As discussed in the office hour, we tried to find the best features by looking at their importance before modeling, not assuming a tree or any other underlying model. Overall the idea was to check mutual information (A. Kraskov, H. Stogbauer and P. Grassberger, “Estimating mutual information”. Phys. Rev. E 69, 2004.) and correlation to determine the important feature. Since we are training a global model we wanted to check for the global correlation and mutual information first:

In [ ]:
#plot global correlation heatmap and show the order of features based on global correlation
#Stocksplits does not have any entries in our sample--> empty/NaN
df_numeric = df_fs[numerical_cols]
plt.figure(figsize=(35, 35))
sns.heatmap(df_numeric.corr(), annot=True,fmt=".2f", cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

corr_matrix = df_numeric.corr()
target_corr = corr_matrix['Close' ].drop('Close').sort_values(ascending=False)
# print(target_corr)

plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix[['Close']].sort_values(by='Close' ,ascending=False),annot=True, cmap='coolwarm',linewidths=0.5)
plt.title(f"Correlation with '{'Close'}'",fontsize=14)

We see low absolute correlation of  Dividends, StockSplits, difference, returns, volumne and number of days since last entry witht the close price. We can reasonably assume low correlation aswell for the signal for stock splits and dividends.

In [ ]:
#do the same as above but with mutual information
from sklearn.feature_selection import mutual_info_regression

df_numeric = df_fs[numerical_cols]
df_numeric = df_numeric.dropna() #mutual information cannot handle Nans --> drop the max 100 rows for each ticker with Nans
print('we dropped:',df_fs.shape[0]-df_numeric.shape[0],'rows with Nans')

X = df_numeric.drop(columns=['Close'])
y = df_numeric['Close']

mi_scores = mutual_info_regression(X, y, random_state=0)

mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mi_scores})
mi_df = mi_df.sort_values(by='Mutual Information', ascending=False).set_index('Feature')


# print(mi_df)

plt.figure(figsize=(8, 12))
sns.heatmap(mi_df[['Mutual Information']], annot=True, fmt=".3f", cmap='viridis', linewidths=0.5)
plt.title(f"Mutual Information with '{'Close'}'", fontsize=14)
plt.show()

As before for correlation we observe low mutual information for dividends, StockSplits, price difference, returns, volumne and number of days since last entry witht the close price. We can reasonably assume low correlation aswell for the signal for stock splits and dividends.

In [ ]:
n=30 #shows the top 30 features per ticker
def analyze_correlation(df_fs, n_features=n):
    numeric_cols = numerical_cols
    if 'Close' not in numeric_cols:
        raise ValueError("'Close' must bein your DataFrame and numeric.")

    if 'Ticker' not in df_fs.columns:
        raise ValueError("'Ticker' must be in your DataFrame.")
    
    results = []
    for ticker, group in df_fs.groupby('Ticker'):
        if len(group) < 2: #group must have more than two entries
            continue

        group_numeric = group[numeric_cols] #only numeric cols
        corr_matrix = group_numeric.corr()
        target_corr = corr_matrix['Close'].drop('Close').sort_values(ascending=False)
        top_features = target_corr.head(n_features)
        
        for rank, (feature_name, correlation) in enumerate(top_features.items(), start=1):
            results.append({'Ticker': ticker,'Feature': feature_name, 'Correlation': correlation, 'Rank': rank
            })

    results_df = pd.DataFrame(results)
    return results_df


results_df = analyze_correlation(df_fs) 

Following a different logic, we assumed that a high mutual information and correlation for a single Stock (Ticker) would result in a good global performance. Therefore we checked the top features for each ticker and plotted their frequency distribution over all tickers:

In [ ]:
unique_features, counts =np.unique(results_df['Feature'],return_counts=True)
feature_distribution = pd.DataFrame({'Feature': unique_features, 'Count': counts})

threshold = 10 #every ticker has its 30 best features in the results_df. we only show the features that are mentioned more than 10 times in the top 30 features per ticker

# Filter features that meet the threshold condition
filtered_distribution = feature_distribution[feature_distribution['Count']>threshold]

plt.figure(figsize=(10, 6))
plt.bar(filtered_distribution['Feature'],filtered_distribution['Count'], color='skyblue')
plt.xlabel('Feature ')
plt.ylabel('Count')
plt.title('Feature Distribution (Filtered by Count Threshold)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
def analyze_mutual_information(df_fs, n_features=n):
    numeric_cols = numerical_cols
    if 'Close' not in numeric_cols:
        raise ValueError("'Close' must bein your DataFrame and numeric.")

    if 'Ticker' not in df_fs.columns:
        raise ValueError("'Ticker' must be in your DataFrame.")
    
    results = []
    for ticker, group in df_fs.groupby('Ticker'):
        if len(group) < 2: #group must have more than two entries
            continue

        group_numeric = group[numeric_cols] #only numeric cols
        group_numeric = group_numeric.dropna() #mutual information cannot handle Nans --> drop the max 100 rows for each ticker with Nans
        X = group_numeric.drop(columns=['Close'])
        y = group_numeric['Close']
        mutual_info_values = mutual_info_regression(X, y, random_state=0)
        mutual_info_dict = dict(zip(X.columns, mutual_info_values))
        top_features = pd.Series(mutual_info_values, index=X.columns).sort_values(ascending=False).head(n_features)

        
        for rank, (feature_name, mi) in enumerate(top_features.items(), start=1):
            results.append({'Ticker': ticker,'Feature': feature_name, 'Mutual Information': mi, 'Rank': rank
            })

    results_df = pd.DataFrame(results)
    return results_df


mi_results_df = analyze_mutual_information(df_fs) 

In [ ]:
unique_features, counts = np.unique(mi_results_df['Feature'], return_counts=True)
feature_distribution = pd.DataFrame({'Feature': unique_features, 'Count': counts})

threshold = 10 #every ticker has its 30 best features in the results_df. we only show the features that are mentioned more than 10 times in the top 30 features per ticker

filtered_distribution = feature_distribution[feature_distribution['Count'] > threshold]

plt.figure(figsize=(10, 6))
plt.bar(filtered_distribution['Feature'], filtered_distribution['Count'], color='skyblue')
plt.xlabel('Feature')
plt.ylabel('Count')
plt.title('Feature Distribution (Filtered by Count Threshold)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Putting it all together we wanted to indtroduce a metric that is equally weighting the global and ticker specific correlation and mutual information values and return the "importance" of the individual feature:

In [ ]:
def select_features_with_combined_score(global_corr_series, global_mi_df, df_fs, numerical_cols, n_features=30, weights=None):
    if weights is None:
        weights = {
            'global_corr': 0.25,
            'global_mi': 0.25,
            'local_corr': 0.25,
            'local_mi': 0.25
        }

    global_corr_df = pd.DataFrame(global_corr_series)
    global_corr_df.columns = ['Correlation']
    global_corr_df['Norm_Corr'] = MinMaxScaler().fit_transform(global_corr_df[['Correlation']])

    # Global
    global_mi_df['Norm_MI'] = MinMaxScaler().fit_transform(global_mi_df[['Mutual Information']])

    #per ticker
    corr_results_df = analyze_correlation(df_fs, n_features=n_features)
    mi_results_df = analyze_mutual_information(df_fs, n_features=n_features)

    local_corr_norm = pd.DataFrame()
    local_mi_norm = pd.DataFrame()
    
    for feature, group in corr_results_df.groupby('Feature'):
        local_corr_norm.loc[feature, 'Corr_Mean'] = group['Correlation'].mean()

    local_corr_norm['Norm_Corr'] = MinMaxScaler().fit_transform(local_corr_norm[['Corr_Mean']])
    
        
    for feature, group in mi_results_df.groupby('Feature'):
            local_mi_norm.loc[feature, 'MI_Mean'] = group['Mutual Information'].mean()

    local_mi_norm['Norm_MI'] = MinMaxScaler().fit_transform(local_mi_norm[['MI_Mean']])

    features = global_corr_df.index

    results = []

    for feature in features:
        gc = global_corr_df.loc[feature,'Norm_Corr'] if feature in global_corr_df.index else 0
        gm = global_mi_df.loc[feature, 'Norm_MI'] if feature in global_mi_df.index else 0
        lc = local_corr_norm.loc[feature,'Norm_Corr'] if feature in local_corr_norm.index else 0
        lm = local_mi_norm.loc[feature, 'Norm_MI'] if feature in local_mi_norm.index else 0

        final_score = (
            weights['global_corr'] * gc +
            weights['global_mi'] * gm +
            weights['local_corr'] * lc +
            weights['local_mi'] * lm
        )

        results.append((feature, final_score))

    results.sort(key=lambda x: x[1], reverse=True)

    return results

As in the individual and global correlation and mututal information results, we observe almost the same order of features with the last Close Price on the first place, the weighted close priced in order until a lookback of 10. After that we see weighted means and lagged closed prices with higher lookback periods. After that we observe std with high lookback periods and the Day_Number.

In [ ]:
top_features = select_features_with_combined_score(
    target_corr,
    mi_df,
    df_fs,
    numerical_cols,
    n_features=len(numerical_cols)
)

for feature, score in top_features[:10]:
    print(f"{feature}: {score:.4f}")

As a result we decided to include the weighted means and lagged Close prices until a lookback period/lag of 10, plus a short/medium/ and long term weighted mean and std. On top we included the all time features and the catgeorical features Ticker, Industry and Country.

In [ ]:
# feature selection is done with visualization, correlation and mutual information
df = df[[ 'Ticker', 'Industry_Tag', 'Country',
       'Year', 'Month', 'Day', 'DayOfWeek', 
       'Close_Lag_1', 'Close_Lag_2','Close_Lag_3', 'Close_Lag_4', 'Close_Lag_5','Close_Lag_10',
       'Weighted_Mean_2', 'Weighted_Mean_3', 'Weighted_Mean_4','Weighted_Mean_5', 'Weighted_Mean_10',
       'Weighted_Mean_30', 'Weighted_Std_30',
       'Weighted_Mean_60', 'Weighted_Std_60',
       'Weighted_Mean_100','Weighted_Std_100',
       'Day_Number','Close']] 

In [ ]:
# df.to_csv('pr13_final_selected.csv', index=False)

## Modeling
You should implement AT LEAST FIVE approaches we covered, and tune of at least two hyperparameters of each approach.
Do not forget that you should split your data.
You should do model selection and tuning using cross validation on the train set, avoiding data leakage.
Explain and justify what is the metric you are using for model selection and tuning. If your data is imbalanced, consider using techniques for data balancing.

Separately, you should train a neural network. Visualize the training and validation loss. Discuss the network performance

In model selection, make sure when you compare different models and approaches that you compare them on the same dataset, though different transformations could be applied to the comparison dataset.

In [ ]:
def split_data_by_time(df,test_percentage=0.2):
    df = df.sort_values(by='Day_Number')
    n_samples = df.shape[0]
    test_size =int(n_samples *test_percentage)
    train_df =df.iloc[:-test_size] 
    test_df =df.iloc[-test_size:]

    return train_df, test_df

In [ ]:
def create_X_y(df):
    X = df.drop(columns=["Close"])
    y = df["Close"]

    return X,y

# Here we take only data from 2020 onwards

In [ ]:
#we assume that the last years are the most important ones and very old data just introduces more noise. We have included a period of growth, before and after covid, and we observe the behaviour of a crisis (COVID).
df = df[df['Day_Number']>=7304] #first day of 2020 --> 19879 rows 
split_day = 7304

#set X,y for train and test set + set cv sets since they all have to be equal for all models to compare them in the end
train_df, test_df = split_data_by_time(df)
X_train, y_train = create_X_y(train_df)
X_test, y_test = create_X_y(test_df)

numerical_cols = df.select_dtypes(include='float64').columns.tolist() 
numerical_cols.append('Day_Number')


all_columns = df.columns.tolist()
categorical_cols = list(set(all_columns) - set(numerical_cols))

numerical_cols.remove("Close")


# print(categorical_cols)
# print(numerical_cols)
df = df.sort_values(by='Day_Number')
cv = TimeSeriesSplit(n_splits=5)


## Linear Models

### Ridge

### Lasso

## Tree Models

### XGBoost

In [ ]:
xgb_numeric_preprocessor = Pipeline([ 
    ("scaler", StandardScaler())
])                  

xgb_categorical_preprocessor = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False, drop='first',handle_unknown='infrequent_if_exist'))
])

xgb_preprocessor = ColumnTransformer([
    ("numerical",xgb_numeric_preprocessor, numerical_cols),
    ("categorical",xgb_categorical_preprocessor, categorical_cols)
    
], remainder='passthrough')


xgb_pipe = Pipeline([
    ('preprocessor',xgb_preprocessor),
    ('regressor',  XGBRegressor(random_state=0))
])


xgb_param_grid = {
    'preprocessor__numerical__scaler':[StandardScaler(), 'passthrough'],
    'regressor__learning_rate': sp_uniform(loc=0.01, scale=0.2),
    'regressor__max_depth': [2,3, 5, 7, 10],
    'regressor__n_estimators': [300,500,700,1000,1200]
}

cv = TimeSeriesSplit(n_splits=5)

xgb_grid_search = RandomizedSearchCV(xgb_pipe, param_distributions=xgb_param_grid, cv=cv, random_state=0, n_jobs=-1,n_iter=15,error_score='raise' ,scoring='neg_root_mean_squared_error')
xgb_grid_search.fit(X_train, y_train)



print(xgb_pipe.get_params())
xgb_grid_search.fit(X_train,y_train)
xgb_best_params = xgb_grid_search.best_params_
print('Best parameters:',xgb_best_params)

with open(f'best_xgb_model_{split_day}.pkl', 'wb') as f:
    pickle.dump(xgb_grid_search.best_estimator_, f)

In [ ]:
plt.figure(figsize=(20,20))
plot_tree(xgb_grid_search.best_estimator_.named_steps['regressor'], num_trees=0)
plt.show()

### CatBoost

In [ ]:
cat_numeric_preprocessor = Pipeline([ 
    ("scaler", StandardScaler())
])                  

cat_preprocessor = ColumnTransformer([
    ("numerical", cat_numeric_preprocessor, numerical_cols),
    ("categorical", 'passthrough',categorical_cols)
], remainder='passthrough')

cat_pipe = Pipeline([
    ('preprocessor', cat_preprocessor),
    ('regressor',  CatBoostRegressor(random_state=0, cat_features=list(range(len(numerical_cols), len(numerical_cols) + len(categorical_cols))))) #order changes after preprocessor
])

cat_param_grid = {
    'preprocessor__numerical__scaler':[StandardScaler(), 'passthrough'],
    'regressor__learning_rate': sp_uniform(loc=0.01, scale=0.2),
    'regressor__depth': [3, 5, 7, 10],
    'regressor__iterations': [300, 500, 700, 1000,1200]
}

cv = TimeSeriesSplit(n_splits=5)

cat_grid_search = RandomizedSearchCV(cat_pipe, param_distributions=cat_param_grid, cv=cv, random_state=0, n_jobs=-1, n_iter=15, error_score='raise', scoring='neg_root_mean_squared_error')



print(cat_pipe.get_params())
cat_grid_search.fit(X_train,y_train)
cat_best_params = cat_grid_search.best_params_
print('Best parameters:',cat_best_params)

with open(f'best_cat_model_{split_day}.pkl', 'wb') as f:
    pickle.dump(cat_grid_search.best_estimator_, f)

### Random Forest

In [ ]:
rf_numeric_preprocessor = Pipeline([ 
    ("scaler", StandardScaler())
])                  

rf_categorical_preprocessor = Pipeline([
    ("onehot", OneHotEncoder(sparse_output=False, drop='first',handle_unknown='infrequent_if_exist'))
])  

rf_preprocessor = ColumnTransformer([
    ("numerical", rf_numeric_preprocessor, numerical_cols),
    ("categorical", rf_categorical_preprocessor, categorical_cols)
], remainder='passthrough')

rf_pipe = Pipeline([
    ('preprocessor', rf_preprocessor),
    ('regressor',  RandomForestRegressor(random_state=0))
])

rf_param_grid = {
    'preprocessor__numerical__scaler':[StandardScaler(), 'passthrough'],
    'regressor__n_estimators': [100, 200, 300, 400, 500, 600],
    'regressor__max_depth': [4, 5, 6, 7, None],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
}

cv = TimeSeriesSplit(n_splits=5)

rf_grid_search = RandomizedSearchCV(rf_pipe, param_distributions=rf_param_grid, cv=cv, random_state=0, n_jobs=-1, n_iter=15, error_score='raise', scoring='neg_root_mean_squared_error')

X_train_cleaned = X_train.dropna()
y_train_cleaned = y_train[X_train_cleaned.index]

print(rf_pipe.get_params())
rf_grid_search.fit(X_train_cleaned, y_train_cleaned)
rf_best_params = rf_grid_search.best_params_
print('Best parameters:',rf_best_params)

with open(f'best_rf_model_{split_day}.pkl', 'wb') as f:
    pickle.dump(rf_grid_search.best_estimator_, f)

## Classification Models

### KNN

In [56]:
df_knn = df.copy()
df_knn['target'] = np.log(df_knn['Close'].shift(-1)) - np.log(df_knn['Close'])
df_knn = df_knn.dropna(subset=['target'])
df_knn = df_knn.dropna() #194 rows dropped

train_df_knn, test_df_knn = split_data_by_time(df_knn)
X_train_knn = train_df_knn.drop(columns=['target', 'Close'])
y_train_knn = train_df_knn['target']
X_test_knn = test_df_knn.drop(columns=['target', 'Close'])
y_test_knn = test_df_knn['target']

cat_knn = ['Industry_Tag', 'Country', 'DayOfWeek', 'Month']
num_knn = [
    'Year', 'Day',
    'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3', 'Close_Lag_4', 'Close_Lag_5', 'Close_Lag_10',
    'Weighted_Mean_2', 'Weighted_Mean_3', 'Weighted_Mean_4', 'Weighted_Mean_5', 'Weighted_Mean_10',
    'Weighted_Mean_30', 'Weighted_Std_30', 'Weighted_Mean_60', 'Weighted_Std_60',
    'Weighted_Mean_100', 'Weighted_Std_100', 'Day_Number'
]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_knn),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_knn)
])

knn_pipe = Pipeline([
    ("preprocessing", preprocessor),
    ("knn", KNeighborsRegressor(n_neighbors=5))
])

param_grid = {
    "knn__n_neighbors": list(range(1, 100)),
    "knn__weights": ["uniform", "distance"]
}

cv = TimeSeriesSplit(n_splits=5)

knn_grid_search = GridSearchCV(
    knn_pipe,
    param_grid,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print(knn_pipe.get_params())
knn_grid_search.fit(X_train_knn, y_train_knn)
knn_best_params = knn_grid_search.best_params_
print('Best parameters:', knn_best_params)

{'memory': None, 'steps': [('preprocessing', ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['Year', 'Day', 'Close_Lag_1', 'Close_Lag_2',
                                  'Close_Lag_3', 'Close_Lag_4', 'Close_Lag_5',
                                  'Close_Lag_10', 'Weighted_Mean_2',
                                  'Weighted_Mean_3', 'Weighted_Mean_4',
                                  'Weighted_Mean_5', 'Weighted_Mean_10',
                                  'Weighted_Mean_30', 'Weighted_Std_30',
                                  'Weighted_Mean_60', 'Weighted_Std_60',
                                  'Weighted_Mean_100', 'Weighted_Std_100',
                                  'Day_Number']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Industry_Tag', 'Country', 'DayOfWeek',
                                  'Month'])])), ('knn', KNeighborsRegressor())], 'transform_input': 

## Model evaluation

After selecting your final model, which could be a compromise of performance, interpretability and complexity, you should evaluate its performance on the test set. 
You might have tuned your model using a certain metric, but now you should describe the model performance using all relevant metrics. 
If you have some business insight, why a certain metric is relevant, you should explain it. 
Construct a suitable baseline to benchmark your result and to put them in the context.
Discuss your results, do they seem good enough to be used in practice? If not, what should be improved. Discuss what type of errors is your model making.

In [ ]:
def evaluate_shap_feature_importance(estimator, X):
    print("Calculating SHAP values...")
    explainer = shap.TreeExplainer(estimator.named_steps['regressor'])
    feature_names = estimator.named_steps['preprocessor'].get_feature_names_out()
    X_processed = estimator.named_steps['preprocessor'].transform(X)
    shap_values = explainer.shap_values(X_processed)

    print("Plotting SHAP summary (bar)...")
    shap.summary_plot(shap_values, X_processed, feature_names=feature_names,plot_type="bar")

    print("Plotting SHAP summary (beeswarm)...")
    shap.summary_plot(shap_values, X_processed,feature_names=feature_names, plot_type="dot")



def evaluate_permutation_feature_importance(model, X, y, n_repeats=30, random_state=0):

    print("Calculating Permutation Importance...")
    perm_result =permutation_importance(model, X,y,n_repeats=n_repeats, random_state=random_state, n_jobs=-1)
    sorted_idx = perm_result.importances_mean.argsort()

    plt.figure(figsize=(10, 5))
    plt.barh(X.columns[sorted_idx],perm_result.importances_mean[sorted_idx])
    plt.xlabel("Permutation Importance")
    plt.title("Feature Importance with Permutation")
    plt.show()

In [ ]:
def evaluate_model(model, X_test, y_test, benchmark_column='Close_Lag_1'):

    y_test_pred = model.predict(X_test)
    
    #model metrics
    mse_test =mean_squared_error(y_test, y_test_pred)
    mae_test =mean_absolute_error(y_test, y_test_pred)
    mape_test= mean_absolute_percentage_error(y_test, y_test_pred)
    R_2_test = r2_score(y_test, y_test_pred)
    adj_R2_test = 1- ((1-R_2_test)*(X_test.shape[0] - 1)/(X_test.shape[0]- X_test.shape[1] -1))
    
    #benchmark metrics
    y_pred_benchmark = X_test[benchmark_column]
    mse_benchmark = mean_squared_error(y_test, y_pred_benchmark)
    mae_benchmark = mean_absolute_error(y_test, y_pred_benchmark)
    mape_benchmark = mean_absolute_percentage_error(y_test, y_pred_benchmark)
    R_2_benchmark = r2_score(y_test, y_pred_benchmark)
    adj_R2_benchmark = 1 - ((1 -R_2_benchmark) *(X_test.shape[0] -1) /(X_test.shape[0]-X_test.shape[1] - 1))
    
    print("---- Model Evaluation ----")
    print("Mean Squared Error on test set:",mse_test)
    print("Root Mean Squared Error on test set:",np.sqrt(mse_test))
    print("Mean Absolute Error on test set:",mae_test)
    print("Mean Absolute Percentage Error on test set:",mape_test)
    print("R^2 on test set:", R_2_test)
    print("Adjusted R^2 on test set:", adj_R2_test)
    
    print("---- Benchmark Comparison ----")
    print('Benchmark:',benchmark_column)
    print("Mean Squared Error on benchmark:",mse_benchmark)
    print("Root Mean Squared Error on benchmark:",np.sqrt(mse_benchmark))
    print("Mean Absolute Error on benchmark:",mae_benchmark)
    print("Mean Absolute Percentage Error on benchmark:",mape_benchmark)
    print("R^2 on benchmark:", R_2_benchmark)
    print("Adjusted R^2 on benchmark:", adj_R2_benchmark)

## Model interpretation

Use at least two different techniques for model interpretability. Discuss what are the most important features of your model, and how they impact the model performance. Pick a few examples of errors that your model is making, and check which features lead to thess errors.